In [2]:
from config import API_Key
import pandas as pd
import requests
import json
from citipy import citipy
import numpy as np

In [5]:
url = f"http://api.airvisual.com/v2/city?city=Los Angeles&state=California&country=USA&key={API_Key}"
response=requests.get(url)
data=response.json()
print(json.dumps(data,indent=4,sort_keys=4))

{
    "data": {
        "city": "Los Angeles",
        "country": "USA",
        "current": {
            "pollution": {
                "aqicn": 33,
                "aqius": 73,
                "maincn": "p2",
                "mainus": "p2",
                "ts": "2022-06-16T08:00:00.000Z"
            },
            "weather": {
                "hu": 85,
                "ic": "50n",
                "pr": 1009,
                "tp": 18,
                "ts": "2022-06-16T08:00:00.000Z",
                "wd": 130,
                "ws": 3.09
            }
        },
        "location": {
            "coordinates": [
                -118.2417,
                34.0669
            ],
            "type": "Point"
        },
        "state": "California"
    },
    "status": "success"
}


In [4]:
#call climate classifications from csv by lat/lon
filepath="Climate_Data/ClimateClassifications.csv"
climate_df=pd.read_csv(filepath)
climate_df

,Lat,Lon,Cls
0,-89.75,-179.75,EF
1,-89.75,-179.25,EF
2,-89.75,-178.75,EF
3,-89.75,-178.25,EF
4,-89.75,-177.75,EF
...,...,...,...
92411,83.75,-30.75,ET
92412,83.75,-30.25,ET
92413,83.75,-29.75,ET
92414,83.75,-29.25,ET


In [5]:
#create list of nearest city to each row of climate datadrame
nearest_city=[]
nearest_city_df=climate_df
for i in range(0,len(nearest_city_df)):
    lat=nearest_city_df.loc[i,"Lat"]
    lon=nearest_city_df.loc[i,"Lon"]
    city=citipy.nearest_city(lat,lon)
    nearest_city.append(city.city_name)
    
    


In [6]:
#create new dataframe with column showing nearest city to each lat/lon
climate_cities=nearest_city_df
climate_cities["Nearest City"]=nearest_city
climate_cities
climate_cities.to_csv("Climate_Data/Nearest_Cities.csv",index=False)
climate_cities

NameError: name 'nearest_city_df' is not defined

In [31]:
#Remove all duplicate city names to be left with a list of cities with their climate classifcation
cities_shortlist_df=climate_cities.drop_duplicates(subset=["Nearest City"],keep='first')
cities_shortlist_df=cities_shortlist_df.drop(columns=["Lat","Lon"])
cities_shortlist_df=cities_shortlist_df.reset_index(drop=True)
cities_shortlist_df=cities_shortlist_df.rename(columns={"Nearest City":"City","Cls":"Climate"})
cities_shortlist_df=cities_shortlist_df[["City","Climate"]]
cities_shortlist_df.to_csv("Climate_Data/Nearest_Cities_Shortlist.csv",index=False)
cities_shortlist_df

,City,Climate
0,vaini,EF
1,mataura,EF
2,rikitea,EF
3,punta arenas,EF
4,ushuaia,EF
...,...,...
14939,dikson,ET
14940,qaanaaq,ET
14941,longyearbyen,ET
14942,narsaq,ET


In [4]:
world_cities_df=pd.read_csv("Climate_Data/world_cities.csv")
cols = world_cities_df.select_dtypes(include=[object]).columns
world_cities_df[cols] = world_cities_df[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
world_cities_df['name']=world_cities_df['name'].str.lower()
world_cities_df.head(25)

,name,country,subcountry,geonameid
0,les escaldes,Andorra,Escaldes-Engordany,3040051
1,andorra la vella,Andorra,Andorra la Vella,3041563
2,umm al qaywayn,United Arab Emirates,Umm al Qaywayn,290594
3,ras al-khaimah,United Arab Emirates,Ras al Khaymah,291074
4,khawr fakkan,United Arab Emirates,Ash Shariqah,291696
5,dubai,United Arab Emirates,Dubai,292223
6,dibba al-fujairah,United Arab Emirates,Al Fujayrah,292231
7,dibba al-hisn,United Arab Emirates,Al Fujayrah,292239
8,sharjah,United Arab Emirates,Ash Shariqah,292672
9,ar ruways,United Arab Emirates,Abu Dhabi,292688


In [15]:
cities_shortlist_df=pd.read_csv("Climate_Data/Nearest_Cities_Shortlist.csv")
complete_df=pd.merge(cities_shortlist_df,world_cities_df,left_on='City',right_on='name')
complete_df=complete_df.drop(columns=["name"])
complete_df=complete_df.rename(columns={"City":"city","Climate":"climate"})

complete_df

,city,climate,country,subcountry,geonameid
0,punta arenas,EF,Chile,Magallanes,3874787
1,ushuaia,EF,Argentina,Tierra del Fuego,3833367
2,hermanus,EF,South Africa,Western Cape,3366880
3,kruisfontein,EF,South Africa,Eastern Cape,986717
4,port elizabeth,EF,South Africa,Eastern Cape,964420
...,...,...,...,...,...
7686,severomorsk,Dfc,Russia,Murmansk,496278
7687,norilsk,Dfc,Russia,Krasnoyarskiy,1497337
7688,murmansk,Dfc,Russia,Murmansk,524305
7689,polyarnyy,Dfc,Russia,Murmansk,506763
